# <center>**Notebook get_fights()**</center>

Notebook em jupyter que descreve a função `get_fights()` do scrip ufc_scap() desde o webscraping até os tratamentos realizados para gerar o data frame final.

### Bibliotecas utilizadas

In [1]:
# importando bibliotecas utilizadas no projeto
# Webcraping
import requests 
from bs4 import BeautifulSoup

# Manipulação de dados
import pandas as pd
import numpy as np

# Alteração de diretorio
import os

## Webscraping

A coleta de dados por web scraping para obter informações sobre cada luta apresenta um desafio adicional em comparação com outros casos. Isso ocorre porque os links de acesso estão localizados dentro da tabela de eventos. Portanto, será necessário acessar individualmente cada página de evento para extrair a tabela contendo as lutas e suas respectivas informações.

Primeiramente iremos executar uma função que obtém os links e os eventos.

In [2]:
# Iniciando função

def get_links_fights():
    """
    Função que retorna dicionario de links da tabela de lutas realizada presente na pagina UFCstats e o seu respectivo evento.
    """

    #Link de acesso para pagina 

    URL = "http://www.ufcstats.com/statistics/events/completed?page=all"

    # Realizando o request para o site
    response = requests.get(URL)
    response.raise_for_status()

    # Obtendo a pagina completa em html
    soup = BeautifulSoup(response.content, "html.parser")

    # Encontrando tags <a> com a classe especificada
    links = soup.find_all("a", class_="b-link b-link_style_black")

    # lista para armazenar os eventos
    events = []

    # Lista para armazenar os hrefs
    hrefs = []

    # Salvando os links na lista
    for tag in links:
        hrefs.append(tag["href"])
        events.append(tag.text)

    # Tratando events
    events = [event.split("\n")[1].split("                          ")[-1] for event in events]

    # Criando dicionario a partir das listas
    dict_link = dict(zip(events, hrefs))

    return dict_link

In [ ]:
# Testando a função
dicionario_links = get_links_fights()

dicionario_links

Atrávez desta função, temos os links que levam para as lutas de cada evento.

Antes de desenvolvermos a função que produz o dataframe final, criaremos uma função que coleta o dataframe de lutas de cada evento. Esta função será responsável por processar todas as colunas e retornar esse dataframe, que posteriormente será integrado ao dataframe final consolidado.

In [3]:
def fight_scrap_table(event,link,bruto=False):
    """ 
    Função que realiza o scrap e tratamento de cada evento
    """
    # Link de acesso para pagina
    URL = link

    # Realizando request para o site
    response = requests.get(URL)
    response.raise_for_status()

    # Obtendo pagina complera em html
    soup = BeautifulSoup(response.content, "html.parser")

    # Encontrando tabela
    table = soup.find("table", class_="b-fight-details__table b-fight-details__table_style_margin-top b-fight-details__table_type_event-details js-fight-table")

    # Transformando tabela de html para dataframe pandas ou retornando erro se não encontrar a tabela
    if table:
        # Extraindo cabeçalho
        header = [th.text.strip() for th in table.find('thead').find_all('th')]

        # Extraindo as linhas
        rows = table.find('tbody').find_all('tr')
        table_data = [[col.text.strip() for col in row.find_all('td')] for row in rows]

        # Convertendo em dataframe
        df_bruto = pd.DataFrame(table_data, columns=header) #Obs: Não considera a primeira linha

    else:
        return print("Não foi possível encontrar a tabela desejada na página.")

    # Criando coluna com o evento
    df_bruto["event"] = event

    if bruto:
        return df_bruto
    
    return df_bruto

In [4]:
# Executando a função acima
df_fightes_one = fight_scrap_table("UFC 293: Adesanya vs. Strickland","http://www.ufcstats.com/event-details/ece280745f8727b8")

df_fightes_one

,W/L,Fighter,Kd,Str,Td,Sub,Weight class,Method,Round,Time,event
0,win,Sean Strickland\n \n\n\n\n ...,1\n \n\n \n 0,137\n\n \n\n\n \n ...,0\n \n\n \n 0,0\n \n\n \n 0,Middleweight,U-DEC,5,5:00,UFC 293: Adesanya vs. Strickland
1,win,Alexander Volkov\n \n\n\n\n ...,1\n \n\n \n 0,93\n\n \n\n\n \n ...,0\n \n\n \n 0,1\n \n\n \n 0,Heavyweight,SUB\n\n \n\n Ezekiel Choke,2,4:37,UFC 293: Adesanya vs. Strickland
2,win,Manel Kape\n \n\n\n\n ...,1\n \n\n \n 0,112\n\n \n\n\n \n ...,1\n \n\n \n 0,0\n \n\n \n 0,Flyweight,U-DEC,3,5:00,UFC 293: Adesanya vs. Strickland
3,win,Justin Tafa\n \n\n\n\n ...,1\n \n\n \n 0,11\n\n \n\n\n \n 7,0\n \n\n \n 0,0\n \n\n \n 0,Heavyweight,KO/TKO\n\n \n\n Punch,1,1:22,UFC 293: Adesanya vs. Strickland
4,win,Tyson Pedro\n \n\n\n\n ...,1\n \n\n \n 0,16\n\n \n\n\n \n 8,0\n \n\n \n 0,0\n \n\n \n 0,Light Heavyweight,KO/TKO\n\n \n\n Punch,1,2:12,UFC 293: Adesanya vs. Strickland
5,win,Carlos Ulberg\n \n\n\n\n ...,1\n \n\n \n 0,81\n\n \n\n\n \n ...,1\n \n\n \n 1,1\n \n\n \n 0,Light Heavyweight,SUB\n\n \n\n Rear Naked Choke,3,4:49,UFC 293: Adesanya vs. Strickland
6,win,Chepe Mariscal\n \n\n\n\n ...,0\n \n\n \n 0,38\n\n \n\n\n \n ...,1\n \n\n \n 0,0\n \n\n \n 0,Featherweight,KO/TKO,2,3:19,UFC 293: Adesanya vs. Strickland
7,win,Jamie Mullarkey\n \n\n\n\n ...,0\n \n\n \n 0,88\n\n \n\n\n \n ...,0\n \n\n \n 0,0\n \n\n \n 0,Lightweight,U-DEC,3,5:00,UFC 293: Adesanya vs. Strickland
8,win,Nasrat Haqparast\n \n\n\n\n ...,0\n \n\n \n 0,171\n\n \n\n\n \n ...,0\n \n\n \n 0,0\n \n\n \n 0,Lightweight,U-DEC,3,5:00,UFC 293: Adesanya vs. Strickland
9,win,Charles Radtke\n \n\n\n\n ...,0\n \n\n \n 0,35\n\n \n\n\n \n ...,1\n \n\n \n 0,0\n \n\n \n 0,Welterweight,U-DEC,3,5:00,UFC 293: Adesanya vs. Strickland


Antes de proceguir com os tratamentos, iremos realizar a iteração no dicionario de eventos e links para obter o dataframe completo.

In [5]:
# Criando função get_fights()

def get_fights(bruto=False):
    """
        Função que retorna o dataframe com todas as lutas realizadas e seus respectivos eventos
    """

    # Executando função e obtendo dicionario com eventos e links
    dicionario_event_link = get_links_fights()

    # Criando df_bruto
    df_bruto = pd.DataFrame()

    # Criando loop para realizar a iteração
    for event, link in dicionario_event_link.items():
        # Obtendo dataframe parcial 
        df_parcial = fight_scrap_table(event, link)

        # Criando coluna de evento
        df_parcial['event'] = event

        # Merge no dataframe bruto
        df_bruto = pd.concat([df_bruto, df_parcial], axis=0)



    return df_bruto

In [6]:
# Executando a função
df_bruto = get_fights()

df_bruto

,W/L,Fighter,Kd,Str,Td,Sub,Weight class,Method,Round,Time,event
0,draw\n\n\ndraw,Alexa Grasso\n \n\n\n\n ...,1\n \n\n \n 0,84\n\n \n\n\n \n ...,1\n \n\n \n 4,1\n \n\n \n 1,Women's Flyweight,S-DEC,5,5:00,UFC Fight Night: Grasso vs. Shevchenko 2
1,win,Jack Della Maddalena\n \n\n\n\n ...,0\n \n\n \n 0,105\n\n \n\n\n \n ...,0\n \n\n \n 0,0\n \n\n \n 0,Welterweight,S-DEC,3,5:00,UFC Fight Night: Grasso vs. Shevchenko 2
2,win,Raul Rosas Jr.\n \n\n\n\n ...,1\n \n\n \n 0,18\n\n \n\n\n \n 6,0\n \n\n \n 0,0\n \n\n \n 0,Bantamweight,KO/TKO\n\n \n\n Punches,1,0:54,UFC Fight Night: Grasso vs. Shevchenko 2
3,win,Daniel Zellhuber\n \n\n\n\n ...,0\n \n\n \n 0,36\n\n \n\n\n \n ...,0\n \n\n \n 0,1\n \n\n \n 0,Lightweight,SUB\n\n \n\n Anaconda Choke,2,3:26,UFC Fight Night: Grasso vs. Shevchenko 2
4,win,Kyle Nelson\n \n\n\n\n ...,0\n \n\n \n 0,82\n\n \n\n\n \n ...,0\n \n\n \n 0,0\n \n\n \n 0,Featherweight,U-DEC,3,5:00,UFC Fight Night: Grasso vs. Shevchenko 2
...,...,...,...,...,...,...,...,...,...,...,...
10,win,Orlando Wiet\n \n\n\n\n ...,0\n \n\n \n 0,8\n\n \n\n\n \n 2,0\n \n\n \n 1,0\n \n\n \n 1,Open Weight,KO/TKO,1,2:50,UFC 2: No Way Out
11,win,Frank Hamaker\n \n\n\n\n ...,0\n \n\n \n 0,2\n\n \n\n\n \n 0,1\n \n\n \n 0,3\n \n\n \n 0,Open Weight,SUB\n\n \n\n Keylock,1,4:52,UFC 2: No Way Out
12,win,Johnny Rhodes\n \n\n\n\n ...,0\n \n\n \n 0,11\n\n \n\n\n \n 4,1\n \n\n \n 0,0\n \n\n \n 0,Open Weight,KO/TKO\n\n \n\n Punches,1,12:13,UFC 2: No Way Out
13,win,Patrick Smith\n \n\n\n\n ...,0\n \n\n \n 0,1\n\n \n\n\n \n 1,0\n \n\n \n 0,1\n \n\n \n 0,Open Weight,SUB\n\n \n\n Guillotine Choke,1,0:58,UFC 2: No Way Out


`Criando uma instancia que pode ser executada ao invés da função acima, com o intuito de não ultrapassar os limites de requests da pagina`

`CÓDIGO COMENTADO, EXECUTAR NO LUGAR DA FUNÇÃO ACIMA`

In [8]:
df_bruto = pd.read_csv("../dataframes/df_fights_bruto.csv")

df_bruto.drop(columns=["Unnamed: 0"], inplace=True)

df_bruto

,W/L,Fighter,Kd,Str,Td,Sub,Weight class,Method,Round,Time,event
0,draw\n\n\ndraw,Alexa Grasso\n \n\n\n\n ...,1\n \n\n \n 0,84\n\n \n\n\n \n ...,1\n \n\n \n 4,1\n \n\n \n 1,Women's Flyweight,S-DEC,5,5:00,UFC Fight Night: Grasso vs. Shevchenko 2
1,win,Jack Della Maddalena\n \n\n\n\n ...,0\n \n\n \n 0,105\n\n \n\n\n \n ...,0\n \n\n \n 0,0\n \n\n \n 0,Welterweight,S-DEC,3,5:00,UFC Fight Night: Grasso vs. Shevchenko 2
2,win,Raul Rosas Jr.\n \n\n\n\n ...,1\n \n\n \n 0,18\n\n \n\n\n \n 6,0\n \n\n \n 0,0\n \n\n \n 0,Bantamweight,KO/TKO\n\n \n\n Punches,1,0:54,UFC Fight Night: Grasso vs. Shevchenko 2
3,win,Daniel Zellhuber\n \n\n\n\n ...,0\n \n\n \n 0,36\n\n \n\n\n \n ...,0\n \n\n \n 0,1\n \n\n \n 0,Lightweight,SUB\n\n \n\n Anaconda Choke,2,3:26,UFC Fight Night: Grasso vs. Shevchenko 2
4,win,Kyle Nelson\n \n\n\n\n ...,0\n \n\n \n 0,82\n\n \n\n\n \n ...,0\n \n\n \n 0,0\n \n\n \n 0,Featherweight,U-DEC,3,5:00,UFC Fight Night: Grasso vs. Shevchenko 2
...,...,...,...,...,...,...,...,...,...,...,...
7335,win,Orlando Wiet\n \n\n\n\n ...,0\n \n\n \n 0,8\n\n \n\n\n \n 2,0\n \n\n \n 1,0\n \n\n \n 1,Open Weight,KO/TKO,1,2:50,UFC 2: No Way Out
7336,win,Frank Hamaker\n \n\n\n\n ...,0\n \n\n \n 0,2\n\n \n\n\n \n 0,1\n \n\n \n 0,3\n \n\n \n 0,Open Weight,SUB\n\n \n\n Keylock,1,4:52,UFC 2: No Way Out
7337,win,Johnny Rhodes\n \n\n\n\n ...,0\n \n\n \n 0,11\n\n \n\n\n \n 4,1\n \n\n \n 0,0\n \n\n \n 0,Open Weight,KO/TKO\n\n \n\n Punches,1,12:13,UFC 2: No Way Out
7338,win,Patrick Smith\n \n\n\n\n ...,0\n \n\n \n 0,1\n\n \n\n\n \n 1,0\n \n\n \n 0,1\n \n\n \n 0,Open Weight,SUB\n\n \n\n Guillotine Choke,1,0:58,UFC 2: No Way Out


Assim, temos o dataframe bruto completo com todas as lutas de todos os eventos que possuem dados na pagina UFCstats.

Como se pode notar, o dataframe necessita de várias manipulações antes de estar pronto para ser incorporado ao conjunto final.

O primeiro tratamento será na coluna `Fighter`, considerando que o primeiro lutador foi o que venceu a luta.

In [9]:
# Criando copia do datafram bruto
df_final = df_bruto.copy()

# Tratando coluna Fighter
# Obtendo coluna fighter1
df_final["fighter1"] = df_final["Fighter"].str.split("\n").apply(lambda x: x[0])

# Obtendo coluna fighter2
df_final["fighter2"] = df_final['Fighter'].str.split("              ").apply(lambda x: x[-1])

# Dropando coluna Fighter
df_final = df_final.drop(columns=["Fighter"])

df_final

,W/L,Kd,Str,Td,Sub,Weight class,Method,Round,Time,event,fighter1,fighter2
0,draw\n\n\ndraw,1\n \n\n \n 0,84\n\n \n\n\n \n ...,1\n \n\n \n 4,1\n \n\n \n 1,Women's Flyweight,S-DEC,5,5:00,UFC Fight Night: Grasso vs. Shevchenko 2,Alexa Grasso,Valentina Shevchenko
1,win,0\n \n\n \n 0,105\n\n \n\n\n \n ...,0\n \n\n \n 0,0\n \n\n \n 0,Welterweight,S-DEC,3,5:00,UFC Fight Night: Grasso vs. Shevchenko 2,Jack Della Maddalena,Kevin Holland
2,win,1\n \n\n \n 0,18\n\n \n\n\n \n 6,0\n \n\n \n 0,0\n \n\n \n 0,Bantamweight,KO/TKO\n\n \n\n Punches,1,0:54,UFC Fight Night: Grasso vs. Shevchenko 2,Raul Rosas Jr.,Terrence Mitchell
3,win,0\n \n\n \n 0,36\n\n \n\n\n \n ...,0\n \n\n \n 0,1\n \n\n \n 0,Lightweight,SUB\n\n \n\n Anaconda Choke,2,3:26,UFC Fight Night: Grasso vs. Shevchenko 2,Daniel Zellhuber,Christos Giagos
4,win,0\n \n\n \n 0,82\n\n \n\n\n \n ...,0\n \n\n \n 0,0\n \n\n \n 0,Featherweight,U-DEC,3,5:00,UFC Fight Night: Grasso vs. Shevchenko 2,Kyle Nelson,Fernando Padilla
...,...,...,...,...,...,...,...,...,...,...,...,...
7335,win,0\n \n\n \n 0,8\n\n \n\n\n \n 2,0\n \n\n \n 1,0\n \n\n \n 1,Open Weight,KO/TKO,1,2:50,UFC 2: No Way Out,Orlando Wiet,Robert Lucarelli
7336,win,0\n \n\n \n 0,2\n\n \n\n\n \n 0,1\n \n\n \n 0,3\n \n\n \n 0,Open Weight,SUB\n\n \n\n Keylock,1,4:52,UFC 2: No Way Out,Frank Hamaker,Thaddeus Luster
7337,win,0\n \n\n \n 0,11\n\n \n\n\n \n 4,1\n \n\n \n 0,0\n \n\n \n 0,Open Weight,KO/TKO\n\n \n\n Punches,1,12:13,UFC 2: No Way Out,Johnny Rhodes,David Levicki
7338,win,0\n \n\n \n 0,1\n\n \n\n\n \n 1,0\n \n\n \n 0,1\n \n\n \n 0,Open Weight,SUB\n\n \n\n Guillotine Choke,1,0:58,UFC 2: No Way Out,Patrick Smith,Ray Wizard


Tratando a coluna W/L, que demosntra se a luta teve vitória ou empate.

In [10]:
# Verificando dados desta coluna
df_final["W/L"].value_counts()

win               7207
nc\n\n\nnc          77
draw\n\n\ndraw      56
Name: W/L, dtype: int64

In [11]:
# Tratando a coluna

df_final['w/l'] = df_final["W/L"].apply(lambda x: "nc" if x == "nc\n\n\nnc" else "draw" if x == "draw\n\n\ndraw" else "win")

df_final["w/l"].value_counts()

win     7207
nc        77
draw      56
Name: w/l, dtype: int64

In [12]:
# Retirando coluna "W/L" maiuscula do dataframe

df_final = df_final.drop(columns=["W/L"])

df_final

,Kd,Str,Td,Sub,Weight class,Method,Round,Time,event,fighter1,fighter2,w/l
0,1\n \n\n \n 0,84\n\n \n\n\n \n ...,1\n \n\n \n 4,1\n \n\n \n 1,Women's Flyweight,S-DEC,5,5:00,UFC Fight Night: Grasso vs. Shevchenko 2,Alexa Grasso,Valentina Shevchenko,draw
1,0\n \n\n \n 0,105\n\n \n\n\n \n ...,0\n \n\n \n 0,0\n \n\n \n 0,Welterweight,S-DEC,3,5:00,UFC Fight Night: Grasso vs. Shevchenko 2,Jack Della Maddalena,Kevin Holland,win
2,1\n \n\n \n 0,18\n\n \n\n\n \n 6,0\n \n\n \n 0,0\n \n\n \n 0,Bantamweight,KO/TKO\n\n \n\n Punches,1,0:54,UFC Fight Night: Grasso vs. Shevchenko 2,Raul Rosas Jr.,Terrence Mitchell,win
3,0\n \n\n \n 0,36\n\n \n\n\n \n ...,0\n \n\n \n 0,1\n \n\n \n 0,Lightweight,SUB\n\n \n\n Anaconda Choke,2,3:26,UFC Fight Night: Grasso vs. Shevchenko 2,Daniel Zellhuber,Christos Giagos,win
4,0\n \n\n \n 0,82\n\n \n\n\n \n ...,0\n \n\n \n 0,0\n \n\n \n 0,Featherweight,U-DEC,3,5:00,UFC Fight Night: Grasso vs. Shevchenko 2,Kyle Nelson,Fernando Padilla,win
...,...,...,...,...,...,...,...,...,...,...,...,...
7335,0\n \n\n \n 0,8\n\n \n\n\n \n 2,0\n \n\n \n 1,0\n \n\n \n 1,Open Weight,KO/TKO,1,2:50,UFC 2: No Way Out,Orlando Wiet,Robert Lucarelli,win
7336,0\n \n\n \n 0,2\n\n \n\n\n \n 0,1\n \n\n \n 0,3\n \n\n \n 0,Open Weight,SUB\n\n \n\n Keylock,1,4:52,UFC 2: No Way Out,Frank Hamaker,Thaddeus Luster,win
7337,0\n \n\n \n 0,11\n\n \n\n\n \n 4,1\n \n\n \n 0,0\n \n\n \n 0,Open Weight,KO/TKO\n\n \n\n Punches,1,12:13,UFC 2: No Way Out,Johnny Rhodes,David Levicki,win
7338,0\n \n\n \n 0,1\n\n \n\n\n \n 1,0\n \n\n \n 0,1\n \n\n \n 0,Open Weight,SUB\n\n \n\n Guillotine Choke,1,0:58,UFC 2: No Way Out,Patrick Smith,Ray Wizard,win


O próximo ajuste será nas colunas `kd`, `Str`, `Td` e `Sub`. O valor inicial representa o dado do primeiro lutador, enquanto o valor após todos os "\n" representa o dado do segundo lutador.

In [13]:
# Realizando tratamentos
# Fighter 1
df_final["kd_fighter1"] = df_final['Kd'].str.split("\n").apply(lambda x: x[0])
df_final["str_fighter1"] = df_final['Str'].str.split("\n").apply(lambda x: x[0])
df_final["td_fighter1"] = df_final['Td'].str.split("\n").apply(lambda x: x[0])
df_final["sub_fighter1"] = df_final['Sub'].str.split("\n").apply(lambda x: x[0])

# Fighter 2
df_final["kd_fighter2"] = df_final['Kd'].str.split(" ").apply(lambda x: x[-1])
df_final["str_fighter2"] = df_final['Str'].str.split(" ").apply(lambda x: x[-1])
df_final["td_fighter2"] = df_final['Td'].str.split(" ").apply(lambda x: x[-1])
df_final["sub_fighter2"] = df_final['Sub'].str.split(" ").apply(lambda x: x[-1])

# Dropando colunas após o tratamento
df_final = df_final.drop(columns=["Kd", "Str", "Td", "Sub"])

df_final

,Weight class,Method,Round,Time,event,fighter1,fighter2,w/l,kd_fighter1,str_fighter1,td_fighter1,sub_fighter1,kd_fighter2,str_fighter2,td_fighter2,sub_fighter2
0,Women's Flyweight,S-DEC,5,5:00,UFC Fight Night: Grasso vs. Shevchenko 2,Alexa Grasso,Valentina Shevchenko,draw,1,84,1,1,0,80,4,1
1,Welterweight,S-DEC,3,5:00,UFC Fight Night: Grasso vs. Shevchenko 2,Jack Della Maddalena,Kevin Holland,win,0,105,0,0,0,127,0,0
2,Bantamweight,KO/TKO\n\n \n\n Punches,1,0:54,UFC Fight Night: Grasso vs. Shevchenko 2,Raul Rosas Jr.,Terrence Mitchell,win,1,18,0,0,0,6,0,0
3,Lightweight,SUB\n\n \n\n Anaconda Choke,2,3:26,UFC Fight Night: Grasso vs. Shevchenko 2,Daniel Zellhuber,Christos Giagos,win,0,36,0,1,0,38,0,0
4,Featherweight,U-DEC,3,5:00,UFC Fight Night: Grasso vs. Shevchenko 2,Kyle Nelson,Fernando Padilla,win,0,82,0,0,0,72,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7335,Open Weight,KO/TKO,1,2:50,UFC 2: No Way Out,Orlando Wiet,Robert Lucarelli,win,0,8,0,0,0,2,1,1
7336,Open Weight,SUB\n\n \n\n Keylock,1,4:52,UFC 2: No Way Out,Frank Hamaker,Thaddeus Luster,win,0,2,1,3,0,0,0,0
7337,Open Weight,KO/TKO\n\n \n\n Punches,1,12:13,UFC 2: No Way Out,Johnny Rhodes,David Levicki,win,0,11,1,0,0,4,0,0
7338,Open Weight,SUB\n\n \n\n Guillotine Choke,1,0:58,UFC 2: No Way Out,Patrick Smith,Ray Wizard,win,0,1,0,1,0,1,0,0


A próxima coluna que necessita de tratamento é a coluna Method.

In [14]:
# Tratando a coluna Method
df_final["method"] = df_final["Method"].str.replace("\n","").str.replace("             ","").str.replace(" ","_")

# Dropando coluna Method
df_final = df_final.drop(columns=["Method"])

df_final

,Weight class,Round,Time,event,fighter1,fighter2,w/l,kd_fighter1,str_fighter1,td_fighter1,sub_fighter1,kd_fighter2,str_fighter2,td_fighter2,sub_fighter2,method
0,Women's Flyweight,5,5:00,UFC Fight Night: Grasso vs. Shevchenko 2,Alexa Grasso,Valentina Shevchenko,draw,1,84,1,1,0,80,4,1,S-DEC
1,Welterweight,3,5:00,UFC Fight Night: Grasso vs. Shevchenko 2,Jack Della Maddalena,Kevin Holland,win,0,105,0,0,0,127,0,0,S-DEC
2,Bantamweight,1,0:54,UFC Fight Night: Grasso vs. Shevchenko 2,Raul Rosas Jr.,Terrence Mitchell,win,1,18,0,0,0,6,0,0,KO/TKO_Punches
3,Lightweight,2,3:26,UFC Fight Night: Grasso vs. Shevchenko 2,Daniel Zellhuber,Christos Giagos,win,0,36,0,1,0,38,0,0,SUB_Anaconda_Choke
4,Featherweight,3,5:00,UFC Fight Night: Grasso vs. Shevchenko 2,Kyle Nelson,Fernando Padilla,win,0,82,0,0,0,72,0,0,U-DEC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7335,Open Weight,1,2:50,UFC 2: No Way Out,Orlando Wiet,Robert Lucarelli,win,0,8,0,0,0,2,1,1,KO/TKO
7336,Open Weight,1,4:52,UFC 2: No Way Out,Frank Hamaker,Thaddeus Luster,win,0,2,1,3,0,0,0,0,SUB_Keylock
7337,Open Weight,1,12:13,UFC 2: No Way Out,Johnny Rhodes,David Levicki,win,0,11,1,0,0,4,0,0,KO/TKO_Punches
7338,Open Weight,1,0:58,UFC 2: No Way Out,Patrick Smith,Ray Wizard,win,0,1,0,1,0,1,0,0,SUB_Guillotine_Choke


O próximo tratamento será renomear as colunas com letra maiusculas.

In [15]:
# Renomeando as colunas
df_final = df_final.rename(columns={"Weight class": "weight_class", "Round":"round", "Time":"time"})

df_final

,weight_class,round,time,event,fighter1,fighter2,w/l,kd_fighter1,str_fighter1,td_fighter1,sub_fighter1,kd_fighter2,str_fighter2,td_fighter2,sub_fighter2,method
0,Women's Flyweight,5,5:00,UFC Fight Night: Grasso vs. Shevchenko 2,Alexa Grasso,Valentina Shevchenko,draw,1,84,1,1,0,80,4,1,S-DEC
1,Welterweight,3,5:00,UFC Fight Night: Grasso vs. Shevchenko 2,Jack Della Maddalena,Kevin Holland,win,0,105,0,0,0,127,0,0,S-DEC
2,Bantamweight,1,0:54,UFC Fight Night: Grasso vs. Shevchenko 2,Raul Rosas Jr.,Terrence Mitchell,win,1,18,0,0,0,6,0,0,KO/TKO_Punches
3,Lightweight,2,3:26,UFC Fight Night: Grasso vs. Shevchenko 2,Daniel Zellhuber,Christos Giagos,win,0,36,0,1,0,38,0,0,SUB_Anaconda_Choke
4,Featherweight,3,5:00,UFC Fight Night: Grasso vs. Shevchenko 2,Kyle Nelson,Fernando Padilla,win,0,82,0,0,0,72,0,0,U-DEC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7335,Open Weight,1,2:50,UFC 2: No Way Out,Orlando Wiet,Robert Lucarelli,win,0,8,0,0,0,2,1,1,KO/TKO
7336,Open Weight,1,4:52,UFC 2: No Way Out,Frank Hamaker,Thaddeus Luster,win,0,2,1,3,0,0,0,0,SUB_Keylock
7337,Open Weight,1,12:13,UFC 2: No Way Out,Johnny Rhodes,David Levicki,win,0,11,1,0,0,4,0,0,KO/TKO_Punches
7338,Open Weight,1,0:58,UFC 2: No Way Out,Patrick Smith,Ray Wizard,win,0,1,0,1,0,1,0,0,SUB_Guillotine_Choke


Reordenando o dataframe.

In [16]:
# Reorganizando o dataframe
df_final = df_final[["event", "weight_class", "w/l","fighter1", "fighter2",
                                  "kd_fighter1", "kd_fighter2", "str_fighter1", "str_fighter2",
                                  "td_fighter1", "td_fighter2", "sub_fighter1", "sub_fighter2",
                                  "method", "round", "time"]]


df_final

,event,weight_class,w/l,fighter1,fighter2,kd_fighter1,kd_fighter2,str_fighter1,str_fighter2,td_fighter1,td_fighter2,sub_fighter1,sub_fighter2,method,round,time
0,UFC Fight Night: Grasso vs. Shevchenko 2,Women's Flyweight,draw,Alexa Grasso,Valentina Shevchenko,1,0,84,80,1,4,1,1,S-DEC,5,5:00
1,UFC Fight Night: Grasso vs. Shevchenko 2,Welterweight,win,Jack Della Maddalena,Kevin Holland,0,0,105,127,0,0,0,0,S-DEC,3,5:00
2,UFC Fight Night: Grasso vs. Shevchenko 2,Bantamweight,win,Raul Rosas Jr.,Terrence Mitchell,1,0,18,6,0,0,0,0,KO/TKO_Punches,1,0:54
3,UFC Fight Night: Grasso vs. Shevchenko 2,Lightweight,win,Daniel Zellhuber,Christos Giagos,0,0,36,38,0,0,1,0,SUB_Anaconda_Choke,2,3:26
4,UFC Fight Night: Grasso vs. Shevchenko 2,Featherweight,win,Kyle Nelson,Fernando Padilla,0,0,82,72,0,0,0,0,U-DEC,3,5:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7335,UFC 2: No Way Out,Open Weight,win,Orlando Wiet,Robert Lucarelli,0,0,8,2,0,1,0,1,KO/TKO,1,2:50
7336,UFC 2: No Way Out,Open Weight,win,Frank Hamaker,Thaddeus Luster,0,0,2,0,1,0,3,0,SUB_Keylock,1,4:52
7337,UFC 2: No Way Out,Open Weight,win,Johnny Rhodes,David Levicki,0,0,11,4,1,0,0,0,KO/TKO_Punches,1,12:13
7338,UFC 2: No Way Out,Open Weight,win,Patrick Smith,Ray Wizard,0,0,1,1,0,0,1,0,SUB_Guillotine_Choke,1,0:58


Este é o dataframe final e tratado que a função get_fights() retorna.

Uma implementação para essa função é a alternativa de retornar o event_id em vez do nome do evento. Essa mudança visa facilitar a integração com bancos de dados relacionais. Para executar a função com esta feature, chamar id=True.

Por padrão a função possui id=True e retorna as chaves ao invés do nome do evento.

In [17]:
# Utilizando o script ufc_scrap 
# Primeiro é necessario alterar momentaneamente o diretorio
os.chdir("../")
import ufc_scrap as ufc

# Criando dataframe com as primary keys da tabela events()
df_event_key = ufc.get_events()[["event", "event_id"]]

# Retornando para diretorio normal
os.chdir("documentacao_notebooks")


df_event_key


,event,event_id
0,UFC Fight Night: Fiziev vs. Gamrot,EVE0664
1,UFC Fight Night: Grasso vs. Shevchenko 2,EVE0663
2,UFC 293: Adesanya vs. Strickland,EVE0662
3,UFC Fight Night: Gane vs. Spivac,EVE0661
4,UFC Fight Night: Holloway vs. The Korean Zombie,EVE0660
...,...,...
659,UFC 6: Clash of the Titans,EVE0005
660,UFC 5: The Return of the Beast,EVE0004
661,UFC 4: Revenge of the Warriors,EVE0003
662,UFC 3: The American Dream,EVE0002


In [18]:
# A partir do dataframe podemos realizar um merge com o dataframe

# Left join
df_final = df_final.merge(df_event_key, on="event", how="left")

# Dropando coluna event
df_final = df_final.drop(columns=["event"])

# Reorganizando
cols = ["event_id"] + [col for col in df_final.columns.to_list() if col != "event_id"]
df_final = df_final[cols]

# Visualizando mudanças

df_final


,event_id,weight_class,w/l,fighter1,fighter2,kd_fighter1,kd_fighter2,str_fighter1,str_fighter2,td_fighter1,td_fighter2,sub_fighter1,sub_fighter2,method,round,time
0,EVE0663,Women's Flyweight,draw,Alexa Grasso,Valentina Shevchenko,1,0,84,80,1,4,1,1,S-DEC,5,5:00
1,EVE0663,Welterweight,win,Jack Della Maddalena,Kevin Holland,0,0,105,127,0,0,0,0,S-DEC,3,5:00
2,EVE0663,Bantamweight,win,Raul Rosas Jr.,Terrence Mitchell,1,0,18,6,0,0,0,0,KO/TKO_Punches,1,0:54
3,EVE0663,Lightweight,win,Daniel Zellhuber,Christos Giagos,0,0,36,38,0,0,1,0,SUB_Anaconda_Choke,2,3:26
4,EVE0663,Featherweight,win,Kyle Nelson,Fernando Padilla,0,0,82,72,0,0,0,0,U-DEC,3,5:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7335,EVE0001,Open Weight,win,Orlando Wiet,Robert Lucarelli,0,0,8,2,0,1,0,1,KO/TKO,1,2:50
7336,EVE0001,Open Weight,win,Frank Hamaker,Thaddeus Luster,0,0,2,0,1,0,3,0,SUB_Keylock,1,4:52
7337,EVE0001,Open Weight,win,Johnny Rhodes,David Levicki,0,0,11,4,1,0,0,0,KO/TKO_Punches,1,12:13
7338,EVE0001,Open Weight,win,Patrick Smith,Ray Wizard,0,0,1,1,0,0,1,0,SUB_Guillotine_Choke,1,0:58


E assim  temos o dataframe final com os event_id ao invés do nome do evento.

### Primary Key

A primary key do df_fights ira conter informações do evento em que a luta aconteceu e sua ordem de importancia no card, seguindo o padrão abaixo:

$$FEEEEXX$$

- `F` : Letra que representa que é uma luta.
- `EEEE`: O número do evento que a luta aconteceu ou irá acontecer.
- `XX` : Ordem da luta no dia. 

In [19]:
# Função que cria primary keys 

def fights_key(df):
    # Cópia do dataframe 
    df_process = df[["event_id"]].copy()

    # Numero de lutas de cada evento
    df_aux = df_process.groupby("event_id").size()
    
    # Reordenando dataframe
    df_aux = df_aux.iloc[::-1].reset_index(drop=True)
    aux_list = df_aux.to_list()
    
    # Número de chave
    key_list = [[x for x in range(1, cont + 1)] for cont in aux_list]
    
    # Iterando para primary_keys final
    primay_keys_number = []
    for i in range(0, len(key_list)):
        for num in key_list[i]:
            primay_keys_number.append(num)

    # Numeros das lutas tratado 
    primay_keys_number = [str(num) if len(str(num)) == 2 else "0" + str(num) for num in primay_keys_number]

    # Adicionando coluna no dataframe de processamento
    df_process["key_fight_number"] = primay_keys_number

    # Separando numero do evento
    df_process["key_event_number"] = df_process["event_id"].str[3:]

    # Criando coluna final das keys
    df_process["fights_primary_key"] = "F" + df_process["key_event_number"] + df_process["key_fight_number"]

    # Adicionando ao dataframe final
    df_final = df.copy()
    df_final["fight_id"] = df_process["fights_primary_key"]

    return df_final


fights_key(df_final).head(15)

,event_id,weight_class,w/l,fighter1,fighter2,kd_fighter1,kd_fighter2,str_fighter1,str_fighter2,td_fighter1,td_fighter2,sub_fighter1,sub_fighter2,method,round,time,fight_id
0,EVE0663,Women's Flyweight,draw,Alexa Grasso,Valentina Shevchenko,1,0,84,80,1,4,1,1,S-DEC,5,5:00,F066301
1,EVE0663,Welterweight,win,Jack Della Maddalena,Kevin Holland,0,0,105,127,0,0,0,0,S-DEC,3,5:00,F066302
2,EVE0663,Bantamweight,win,Raul Rosas Jr.,Terrence Mitchell,1,0,18,6,0,0,0,0,KO/TKO_Punches,1,0:54,F066303
3,EVE0663,Lightweight,win,Daniel Zellhuber,Christos Giagos,0,0,36,38,0,0,1,0,SUB_Anaconda_Choke,2,3:26,F066304
4,EVE0663,Featherweight,win,Kyle Nelson,Fernando Padilla,0,0,82,72,0,0,0,0,U-DEC,3,5:00,F066305
5,EVE0663,Women's Strawweight,win,Loopy Godinez,Elise Reed,1,0,21,3,5,0,6,0,SUB_Rear_Naked_Choke,2,3:38,F066306
6,EVE0663,Middleweight,win,Roman Kopylov,Josh Fremd,1,0,49,38,0,0,0,0,KO/TKO_Punch,2,4:44,F066307
7,EVE0663,Flyweight,nc,Edgar Chairez,Daniel Lacerda,0,0,12,4,0,1,1,0,Overturned,1,3:47,F066308
8,EVE0663,Women's Flyweight,win,Tracy Cortez,Jasmine Jasudavicius,0,0,103,77,0,2,0,0,U-DEC,3,5:00,F066309
9,EVE0663,Lightweight,win,Charlie Campbell,Alex Reyes,1,0,43,14,0,0,0,0,KO/TKO_Punches,1,3:38,F066310


Com isso, temos a criação das primary_keys para cada luta do dataframe.

## Atualizações

Proposta de atualização, criar instancia que verifica se é necessario realizar o request pro site ou apenas retorna os dataframes já salvos no computador.